In [2]:
cd("G:\\My Drive\\_MOPJ\\Julia\\ROM4SM\\")

In [5]:
readdir()

6-element Array{String,1}:
 ".git"
 "LICENSE"
 "POD4Frame.ipynb"
 "POD4Frame.jl"
 "POD4Vectors.jl"
 "img"

# Treillis
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_LBC.png" alt="Drawing" style="width: 1000px;"/>


# Fonction Matrice de raideur élémentaire pour un élément de barre plan

In [1]:
function trstiff(coo,E,A)
# Function that computes stiffness matrix for truss element
# (file name trstiff.m)
L=sqrt((coo[2,1]-coo[1,1])^2+(coo[2,2]-coo[1,2])^2)
C=(coo[2,1]-coo[1,1])/L
S=(coo[2,2]-coo[1,2])/L
# Stiffness matrix
STF=E*A/L*[C^2 S*C -C^2 -S*C;
S*C S^2 -S*C -S^2;
-C^2 -S*C C^2 S*C;
-S*C -S^2 S*C S^2];
return(STF)
end

trstiff (generic function with 1 method)

In [1]:
# Main program for truss structure
# Coordinates - units L[mm]
function truss_str(E,A)
COO=[0 0; 2 0; 4 0 ;1 1; 3 1; -1 1; 5 1]
COO=1000*COO
DOF=size(COO,1)*2
# Elements
ELM=[1 2; 2 3; 6 4; 4 5; 5 7; 1 6; 1 4; 2 4; 2 5; 3 5; 3 7]
# Properties (units F[N], L[mm])
#E=125000
#A=6
# Assembling of stiffness matrix
MSTF=zeros(DOF,DOF)
X=zeros(2,2)
for i in 1:size(ELM,1)
X[1,:]=COO[ELM[i,1],:]
X[2,:]=COO[ELM[i,2],:]
STF=trstiff(X,E,A)
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,1:2]
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,1:2]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,3:4]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,3:4]
end
# Constrains
con=[12 13 14 15] # constrained DOF
CDOF=size(con,2)
# Forces
dF=zeros(DOF,1)
dF[2]=-5000
dF[6]=-2000
# Reduced stiffness matrix & force vector
RSTF=MSTF[1:DOF-CDOF,1:DOF-CDOF]
dFR=dF[1:DOF-CDOF]
# Solving for displacements
d=inv(RSTF)*dFR
# End of main program
return(d)
end

truss_str (generic function with 1 method)